# Human Activity Recognition from Wearable Inertial Sensor Networks Part 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Table of Contents
 
- [Import library and define visualization parameters](#Import-library-and-define-visualization-parameters)
- [Declaration of functions used](#Declaration-of-functions-used)
- [Import data](#Import-data)
- [Feature engineering](#Feature-engineering)
- [Save data in pickle format in google drive](#Save-data-in-pickle-format-in-google-drive)

# Import library and define visualization parameters

In [ ]:
from __future__ import print_function
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
%matplotlib inline
from math import pi
import numpy as np
import pandas as pd
from pandas import DataFrame
from typing import List
import seaborn as sns
from scipy import stats

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

import warnings 
warnings.filterwarnings('ignore')

# Display several outputs on a notebook output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Define certain standard parameters at the outset
pd.options.display.float_format = '{: .4f}'. format
np.set_printoptions(precision=4)
plt.style.use ('ggplot')
# Files
path_to_file = '/content/drive/MyDrive/myproject/outputdata'

# Declaration of functions used

In [ ]:
def mad(x):
  mad = x.mad()
  return mad

In [ ]:
def ran(x):
  range = x.max() - x.min()
  return range

In [ ]:
# Define a function to compute acceleration spherical coordinates
def add_accelero_spherical_coordinates(df:DataFrame) -> DataFrame:
  """Generate normal acceleration, colalitude and longitude from imu data.

  Args:
    df (DataFrame): IMU data required!

  Returns:
    DataFrame: The IMU DataFrame with normal acceleration, colalitude and longitude.
  """

  df['acc_norm'] = np.sqrt((df.acc_rf_x)**2 + (df.acc_rf_y)**2 + (df.acc_rf_z)**2)
  df["colat_acc"] = np.arccos(df["acc_rf_z"]/df["acc_norm"])
  df["long_acc"] = np.arctan2(df["acc_rf_y"],df["acc_rf_x"])

  return df

In [ ]:
# Define a function to compute gyrometer spherical coordinates
def add_gyro_spherical_coordinates(df:DataFrame) -> DataFrame:
  """Generate normal gyrometer, colalitude and longitude from imu data.

  Args:
    df (DataFrame): IMU data required!

  Returns:
    DataFrame: The IMU DataFrame with normal gyrometer, colalitude and longitude.
  """
  
  df['gyro_norm'] = np.sqrt((df.gyro_rf_x)**2 + (df.gyro_rf_y)**2 + (df.gyro_rf_z)**2)
  df["colat_gyro"] = np.arccos(df["gyro_rf_z"]/df["gyro_norm"])
  df["long_gyro"] = np.arctan2(df["gyro_rf_y"],df["gyro_rf_x"])

  return df

In [ ]:
def add_group_index(df:DataFrame, windows:int=3) -> DataFrame:
  """Generate group index with specific windows for data aggregation.

  Args:
    df (DataFrame): raw data required!
    windows (int, default value is 5 s): The aggregation window.

  Returns:
    DataFrame: The raw DataFrame containing group index for aggregration.
  """

  sample_size = int((679073/11544)*windows)
  index = df.index.tolist()
  group_index = np.cumsum([(id % sample_size) == 0 for id in index]).tolist()
  df['group_index'] = group_index

  return df

In [ ]:
def compute_features_stats(df:DataFrame, features_to_stats:List[str] = None) -> DataFrame:
  """Generates the aggregated dataframe with the different statistics on the given variables.

  Args:
    df (DataFrame): The raw DataFrame containing group index for aggregration.
    features_to_stats (List): List of variables whose statistics you want to display after aggregation.

  Returns:
    DataFrame: The aggregated dataframe with the different statistics on the input variables.
  """

  stats_agg = [np.min, np.max, np.mean, np.median, np.std, stats.skew, stats.kurtosis, stats.iqr, stats.median_absolute_deviation, mad, ran]
  dict_agg = {var:stats_agg for var in features_to_stats}
  df_agg = df.groupby(['act','group_index', 'filename']).agg(dict_agg)
  df_agg.columns = ["_".join(x) for x in df_agg.columns.ravel()]
  df_agg = df_agg.reset_index(['act', 'group_index', 'filename'])
  df_agg = df_agg.drop('group_index', axis=1)
  df_agg = df_agg.sort_values("filename")
  df_agg = df_agg.interpolate(method='linear', axis=0).ffill().bfill()

  return df_agg

# Import data

In [ ]:
!cp /content/drive/MyDrive/myproject/outputdata/df_1.pkl /tmp/df_1.pkl

In [ ]:
df = pd.read_pickle('/tmp/df_1.pkl')
df

,acc_rf_x,acc_rf_y,acc_rf_z,gyro_rf_x,gyro_rf_y,gyro_rf_z,act,filename,activity,person
0,-8232.0000,-376.0000,13232.0000,-29.0000,181.0000,-167.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01
1,-8244.0000,-236.0000,13252.0000,-146.0000,157.0000,24.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01
2,-8208.0000,-88.0000,13188.0000,-207.0000,114.0000,33.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01
3,-8144.0000,24.0000,13204.0000,-199.0000,167.0000,-23.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01
4,-8204.0000,28.0000,13220.0000,-258.0000,193.0000,-124.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01
...,...,...,...,...,...,...,...,...,...,...
2111397,-9140.0000,-3028.0000,13632.0000,105.0000,-61.0000,-155.0000,5,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Sitting,13
2111398,-9112.0000,-2996.0000,13644.0000,93.0000,3.0000,-138.0000,5,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Sitting,13
2111399,-9124.0000,-3036.0000,13664.0000,151.0000,-1.0000,-165.0000,5,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Sitting,13
2111400,-9124.0000,-2984.0000,13616.0000,145.0000,-94.0000,-168.0000,5,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Sitting,13


# Feature Engineering

## Acceleros and Gyros Features Engineering

In [ ]:
df = add_accelero_spherical_coordinates(df)
df = add_gyro_spherical_coordinates(df)
df

,acc_rf_x,acc_rf_y,acc_rf_z,gyro_rf_x,gyro_rf_y,gyro_rf_z,act,filename,activity,person,acc_norm,colat_acc,long_acc,gyro_norm,colat_gyro,long_gyro
0,-8232.0000,-376.0000,13232.0000,-29.0000,181.0000,-167.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01,15588.2335,0.5570,-3.0959,247.9738,2.3097,1.7297
1,-8244.0000,-236.0000,13252.0000,-146.0000,157.0000,24.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01,15608.8032,0.5567,-3.1130,215.7336,1.4593,2.3199
2,-8208.0000,-88.0000,13188.0000,-207.0000,114.0000,33.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01,15533.9097,0.5567,-3.1309,238.6085,1.4320,2.6382
3,-8144.0000,24.0000,13204.0000,-199.0000,167.0000,-23.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01,15513.5724,0.5527,3.1386,260.8045,1.6591,2.4434
4,-8204.0000,28.0000,13220.0000,-258.0000,193.0000,-124.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01,15558.7532,0.5554,3.1382,345.2376,1.9382,2.4993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111397,-9140.0000,-3028.0000,13632.0000,105.0000,-61.0000,-155.0000,5,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Sitting,13,16689.5119,0.6150,-2.8217,196.9035,2.4770,-0.5263
2111398,-9112.0000,-2996.0000,13644.0000,93.0000,3.0000,-138.0000,5,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Sitting,13,16678.2282,0.6127,-2.8239,166.4392,2.5483,0.0322
2111399,-9124.0000,-3036.0000,13664.0000,151.0000,-1.0000,-165.0000,5,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Sitting,13,16708.3682,0.6132,-2.8204,223.6672,2.4005,-0.0066
2111400,-9124.0000,-2984.0000,13616.0000,145.0000,-94.0000,-168.0000,5,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Sitting,13,16659.7445,0.6141,-2.8255,241.0083,2.3421,-0.5752


## Aggregate Data by time windows

In [ ]:
df = add_group_index(df)
df

,acc_rf_x,acc_rf_y,acc_rf_z,gyro_rf_x,gyro_rf_y,gyro_rf_z,act,filename,activity,person,acc_norm,colat_acc,long_acc,gyro_norm,colat_gyro,long_gyro,group_index
0,-8232.0000,-376.0000,13232.0000,-29.0000,181.0000,-167.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01,15588.2335,0.5570,-3.0959,247.9738,2.3097,1.7297,1
1,-8244.0000,-236.0000,13252.0000,-146.0000,157.0000,24.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01,15608.8032,0.5567,-3.1130,215.7336,1.4593,2.3199,1
2,-8208.0000,-88.0000,13188.0000,-207.0000,114.0000,33.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01,15533.9097,0.5567,-3.1309,238.6085,1.4320,2.6382,1
3,-8144.0000,24.0000,13204.0000,-199.0000,167.0000,-23.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01,15513.5724,0.5527,3.1386,260.8045,1.6591,2.4434,1
4,-8204.0000,28.0000,13220.0000,-258.0000,193.0000,-124.0000,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Bicycling,01,15558.7532,0.5554,3.1382,345.2376,1.9382,2.4993,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111397,-9140.0000,-3028.0000,13632.0000,105.0000,-61.0000,-155.0000,5,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Sitting,13,16689.5119,0.6150,-2.8217,196.9035,2.4770,-0.5263,11990
2111398,-9112.0000,-2996.0000,13644.0000,93.0000,3.0000,-138.0000,5,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Sitting,13,16678.2282,0.6127,-2.8239,166.4392,2.5483,0.0322,11990
2111399,-9124.0000,-3036.0000,13664.0000,151.0000,-1.0000,-165.0000,5,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Sitting,13,16708.3682,0.6132,-2.8204,223.6672,2.4005,-0.0066,11990
2111400,-9124.0000,-2984.0000,13616.0000,145.0000,-94.0000,-168.0000,5,/content/drive/MyDrive/myproject/data/HuGaDB_v...,Sitting,13,16659.7445,0.6141,-2.8255,241.0083,2.3421,-0.5752,11990


In [ ]:
df = df.astype({"act": np.int64})
df_columns = df.columns.to_list()
features_to_stats = [columns for columns in df_columns if columns not in ('filename','activity','person','group_index','act')]
features_to_stats

['acc_rf_x',
 'acc_rf_y',
 'acc_rf_z',
 'gyro_rf_x',
 'gyro_rf_y',
 'gyro_rf_z',
 'acc_norm',
 'colat_acc',
 'long_acc',
 'gyro_norm',
 'colat_gyro',
 'long_gyro']

In [ ]:
df.dtypes

acc_rf_x        float64
acc_rf_y        float64
acc_rf_z        float64
gyro_rf_x       float64
gyro_rf_y       float64
gyro_rf_z       float64
act               int64
filename         object
activity       category
person           object
acc_norm        float64
colat_acc       float64
long_acc        float64
gyro_norm       float64
colat_gyro      float64
long_gyro       float64
group_index       int64
dtype: object

In [ ]:
df_agg = compute_features_stats(df, features_to_stats)
df_agg

,act,filename,acc_rf_x_amin,acc_rf_x_amax,acc_rf_x_mean,acc_rf_x_median,acc_rf_x_std,acc_rf_x_skew,acc_rf_x_kurtosis,acc_rf_x_iqr,acc_rf_x_median_absolute_deviation,acc_rf_x_mad,acc_rf_x_ran,acc_rf_y_amin,acc_rf_y_amax,acc_rf_y_mean,acc_rf_y_median,acc_rf_y_std,acc_rf_y_skew,acc_rf_y_kurtosis,acc_rf_y_iqr,acc_rf_y_median_absolute_deviation,acc_rf_y_mad,acc_rf_y_ran,acc_rf_z_amin,acc_rf_z_amax,acc_rf_z_mean,acc_rf_z_median,acc_rf_z_std,acc_rf_z_skew,acc_rf_z_kurtosis,acc_rf_z_iqr,acc_rf_z_median_absolute_deviation,acc_rf_z_mad,acc_rf_z_ran,gyro_rf_x_amin,gyro_rf_x_amax,gyro_rf_x_mean,gyro_rf_x_median,gyro_rf_x_std,...,long_acc_std,long_acc_skew,long_acc_kurtosis,long_acc_iqr,long_acc_median_absolute_deviation,long_acc_mad,long_acc_ran,gyro_norm_amin,gyro_norm_amax,gyro_norm_mean,gyro_norm_median,gyro_norm_std,gyro_norm_skew,gyro_norm_kurtosis,gyro_norm_iqr,gyro_norm_median_absolute_deviation,gyro_norm_mad,gyro_norm_ran,colat_gyro_amin,colat_gyro_amax,colat_gyro_mean,colat_gyro_median,colat_gyro_std,colat_gyro_skew,colat_gyro_kurtosis,colat_gyro_iqr,colat_gyro_median_absolute_deviation,colat_gyro_mad,colat_gyro_ran,long_gyro_amin,long_gyro_amax,long_gyro_mean,long_gyro_median,long_gyro_std,long_gyro_skew,long_gyro_kurtosis,long_gyro_iqr,long_gyro_median_absolute_deviation,long_gyro_mad,long_gyro_ran
13180,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,-26828.0000,228.0000,-14573.2955,-15614.0000,5793.7028,0.1629,-0.8552,9196.0000,6473.0316,4944.5560,27056.0000,-5964.0000,8844.0000,1925.3182,1846.0000,2520.9770,-0.1507,0.2721,3458.0000,2748.7404,1996.3631,14808.0000,-716.0000,17512.0000,6885.6818,6854.0000,3032.2412,0.4930,0.7177,3854.0000,2873.2788,2349.0573,18228.0000,-7436.0000,7932.0000,-573.8693,-512.5000,4063.0418,...,2.4858,-1.3142,-0.2451,0.4017,0.1865,2.0664,6.2716,945.3571,23264.5105,10310.5263,10026.8826,5410.1511,0.2343,-0.7897,8611.3435,6434.9378,4575.3236,22319.1535,0.7905,2.4534,1.5761,1.7191,0.3832,0.0089,-1.2411,0.6793,0.5010,0.3500,1.6628,-2.7910,3.0252,-0.1613,0.9271,1.7209,-0.1405,-1.7772,3.2849,1.2820,1.6712,5.8162
13173,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,-32768.0000,2744.0000,-15908.3182,-17936.0000,8476.0718,0.2310,-1.0835,13653.0000,10247.7312,7440.2546,35512.0000,-7160.0000,11156.0000,1906.0909,2280.0000,3006.8416,-0.2840,0.3281,3892.0000,2653.8540,2344.5134,18316.0000,-2660.0000,16468.0000,6917.0682,7444.0000,4221.9786,-0.3141,-0.6804,5915.0000,4581.2340,3497.1772,19128.0000,-8612.0000,6710.0000,-724.3693,-1421.5000,4409.9493,...,2.5117,-1.1491,-0.6286,2.5134,0.2027,2.1566,6.2694,1842.2904,26660.7860,13272.7228,13334.5933,6810.6574,0.0467,-1.1338,11651.8404,8842.4359,5783.4449,24818.4956,0.5558,2.3396,1.5991,1.8195,0.4306,-0.1726,-1.4494,0.7665,0.5667,0.4046,1.7838,-2.7197,2.5692,-0.4510,-1.7267,1.6603,0.1622,-1.8353,3.2286,1.0716,1.6168,5.2889
13174,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,-30940.0000,1836.0000,-16694.3182,-18150.0000,8272.1209,0.4232,-0.8617,13774.0000,9230.6676,7004.7557,32776.0000,-6472.0000,8020.0000,1933.4773,2478.0000,2786.0552,-0.3112,-0.3191,3741.0000,2745.7752,2283.8549,14492.0000,-3380.0000,14924.0000,6404.4545,7276.0000,3980.7550,-0.4040,-0.7550,6252.0000,4237.2708,3361.0393,18304.0000,-8296.0000,6519.0000,333.5682,678.5000,3950.7688,...,2.5034,-1.1666,-0.5756,2.2976,0.1884,2.1428,6.2821,1013.4639,25378.0033,12059.3191,12647.7960,6183.3921,0.0505,-0.9531,9699.8500,7468.2433,5196.0901,24364.5394,0.7882,2.5435,1.5631,1.4746,0.4225,0.2112,-1.2869,0.7831,0.5797,0.3847,1.7553,-2.9461,2.7416,-0.0474,1.0783,1.6228,-0.2627,-1.7239,3.2073,0.7663,1.5615,5.6877
13175,9,/content/drive/MyDrive/myproject/data/HuGaDB_v...,-27988.0000,100.0000,-14594.7045,-15108.0000,7188.4512,0.0694,-1.1602,12289.0000,9310.7280,6313.0522,28088.0000,-4552.0000,9284.0000,1653.3409,1506.0000,2464.6819,0.0466,-0.1943,3227.0000,2378.0904,1963.5834,13836.0000,-1880.0000,11880.0000,5811.5455,6510.0000,3210.8786,-0.3152,-0.9499,5090.0000,3641.2656,2741.2789,13760.0000,-8277.0000,690

# Save data in pickle format in google drive


In [ ]:
df_agg.to_pickle("/tmp/df_2.pkl")

In [ ]:
!cp /tmp/df_2.pkl /content/drive/MyDrive/myproject/outputdata